* Find file location on disk
  * Place under Git
  * Create new IHE-SDC Git repo for SDC Object Model Usage Samples
  * Create a folder for SDC XML files
  * Create "Assmblies" folder for imported assemblies
    * Copy in the latest SDC.Schema dll 
    * Copy in "C:\Program Files\Microsoft Visual Studio\2022\Enterprise\Common7\IDE\PublicAssemblies\Microsoft.VisualStudio.QualityTools.UnitTestFramework.dll" //Microsoft.VisualStudio.TestTools.UnitTesting
    * Create Archive folder for junk notebooks

In [ ]:
#r "./Assemblies/SDC_CodeGeneratorTest.dll"  //SDC.Schema namespace
#r "./Assemblies/Microsoft.VisualStudio.QualityTools.UnitTestFramework.dll"  //unit test framework

using System;
    using System.Runtime;
    using System.IO;

    using Microsoft.VisualStudio.TestTools.UnitTesting;
    using Newtonsoft.Json;
    using SDC.Schema;
    using SDC;
    using System.Data;
    using System.Diagnostics;
    using System.Drawing;
    using System.Linq;
    using System.Xml;

* Read in a large SDC file, and then deserialize as XML

In [ ]:
string path = Path.Combine("C:", "Users", "rmoldwi", "OneDrive", "One Drive Documents", "Visual Studio 2017", "Projects", "SDC Framework", "Core Tests", "Test files", "Breast.Invasive.Staging.359_.CTP9_sdcFDF.xml");
var FD = SDC.Schema.FormDesignType.DeserializeFromXmlPath(path);
string myXML =  SdcSerializer<FormDesignType>.Serialize(FD);
myXML

<?xml version="1.0" encoding="utf-8"?>
<FormDesign xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" order="2431" ID="Breast.Invasive.Bx.Res.359_.CTP9_sdcFDF" baseURI="cap.org" formTitle="Breast Cancer Staging Calculator" filename="Breast.Ca.Stg.Calc.359_CTP9_sdcFDF.xml" fullURI="_baseURI=cap.org&amp;_lineage=Breast.Ca.Stg.Calc.359&amp;_version=.CTP9&amp;_docType=sdcFDF" version="CTP9" lineage="Breast.Ca.Stg.Calc.359" xmlns="urn:ihe:qrph:sdc:2016">
  <Property name="SaveState" order="2432" propName="SaveState" val="New" />
  <Property name="Copyright" type="CAPeCC_static_text" styleClass="copyright" order="2433" propName="Copyright" val="(c) 2018 College of American Pathologists.  All rights reserved.  License required for use." />
  <Property name="GenericHeaderText" type="CAPeCC_static_text" order="2434" propName="GenericHeaderText" val="Breast Cancer Staging Calculator" />
  <Property name="Category" type="CAPeCC_meta" order="2435" pr

## Find SDC node changes compared to the reference template
* Create 2 FormDesign OMs, "newOM" and "refOM"
* Iterate each node in newOM, and locate matcching node in refOM by ID, SGuid and/or name
  * For each newOM node, determine if it's 
    * new, 
    * has a different parent node, 
    * previous sib has changed.
  * Add the changed node's ID/name/sGUID and change type to a dictionary
* Compare each SDC property in newOM with the matching refOM node
* 